# Praca domowa 4

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import math

import dalex as dx
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
import sklearn
import warnings
warnings.filterwarnings('ignore')

In [2]:
apartments = dx.datasets.load_apartments()

In [3]:
apartments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 1 to 1000
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   m2_price           1000 non-null   int64 
 1   construction_year  1000 non-null   int64 
 2   surface            1000 non-null   int64 
 3   floor              1000 non-null   int64 
 4   no_rooms           1000 non-null   int64 
 5   district           1000 non-null   object
dtypes: int64(5), object(1)
memory usage: 54.7+ KB


In [4]:
apartments.district.value_counts() # 10 kolumn

Mokotow        107
Wola           106
Ursus          105
Ursynow        103
Srodmiescie    100
Bemowo          98
Zoliborz        97
Bielany         96
Ochota          96
Praga           92
Name: district, dtype: int64

#### Transformacja kolumny `district` przy użyciu Target Encoder

In [5]:
X = apartments.loc[:, apartments.columns != "m2_price"]
y = apartments.m2_price

from category_encoders import TargetEncoder

tge = TargetEncoder()
X['district'] = tge.fit_transform(X['district'], y)
X

,construction_year,surface,floor,no_rooms,district
1,1953,25,3,1,5182.750000
2,1992,143,9,5,2985.739583
3,1937,56,1,2,2994.804348
4,1995,93,7,3,3956.291667
5,1992,144,6,5,3855.457944
...,...,...,...,...,...
996,1921,44,2,2,5182.750000
997,1921,48,10,2,3048.459184
998,1980,85,3,3,3048.459184
999,1942,36,7,1,3797.948454


In [6]:
from sklearn.model_selection import cross_val_score, cross_validate

### Model performance przy braku skalowania zmiennych

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 45)

Zastosowanie kroswalidacji

In [92]:
svm_base = SVC(verbose = False)
res = cross_validate(svm_base, X_train, y_train, cv=3, scoring=['neg_mean_squared_error', 'r2'])
print("SVC", 'RMSE: {0:0.4f}'. format(np.mean(res['test_neg_mean_squared_error'])))
print("SVC", 'R2: {0:0.4f}'. format(np.mean(res['test_r2'])))

SVC RMSE: -6352.3857
SVC R2: -0.0305


In [17]:
svm_base1 = SVC(verbose = False) 
svm_base1.fit(X_train, y_train)
y_hat = svm_base1.predict(X_test)
print(mean_squared_error(y_test, y_hat))
print(r2_score(y_test, y_hat))

713048.776
0.15633550700141008


## Skalowanie zmiennych

In [18]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC, SVR

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score


scaler = MinMaxScaler()
X1 = scaler.fit_transform(X)

In [19]:
from sklearn.model_selection import train_test_split

X1_train, X1_test, y_train, y_test = train_test_split(X1, y, test_size = 0.25, random_state = 45)

In [20]:
svm_base = SVC(verbose = False)
res = cross_validate(svm_base, X_train, y_train, cv=3, scoring=['neg_mean_squared_error', 'r2'])
print("SVC", 'RMSE: {0:0.4f}'. format(np.mean(res['test_neg_mean_squared_error'])))
print("SVC", 'R2: {0:0.4f}'. format(np.mean(res['test_r2'])))

SVC RMSE: -428321.6040
SVC R2: 0.4677


In [59]:
svm_base2 = SVC(verbose = False)
svm_base2.fit(X1_train, y_train)
y_hat = svm_base2.predict(X1_test)
mse_base = mean_squared_error(y_test, y_hat)
print(mean_squared_error(y_test, y_hat))
print(r2_score(y_test, y_hat))

242591.544
0.7129707267395897


In [75]:
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

### Wnioski: 
- Skalowanie zmiennych zmniejszyło wartość błędu RMSE o ponad 25% w przypadku kroswalidacji na zbiorze treningowym
- W przypadku skalowania zmiennych przy braku kroswalidacji wartość błędu RMSE zmniejszyła się trzykrotnie

# Randomized Search
Początkowe ustawienia parametrów:
- C - z rozkładu jednostajnego na przedziale [0, 5]
- gamma: ['scale', 'auto']
- degree: [2, 3, 4,5,6]

In [27]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

distributions = dict(C=uniform(loc=0, scale=5), gamma = ['scale', 'auto'], degree = [2, 3, 4,5,6])
clf = RandomizedSearchCV(svm_base, param_distributions = distributions, random_state=0, cv=3, n_iter = 20)
search = clf.fit(X1_train, y_train)

In [28]:
search_res = pd.DataFrame(search.cv_results_)

# Top 5
for i in range(5):
    print(search_res.iloc[i, 7])

{'C': 2.7440675196366238, 'degree': 2, 'gamma': 'auto'}
{'C': 4.289728088113784, 'degree': 3, 'gamma': 'auto'}
{'C': 3.2294705653332807, 'degree': 6, 'gamma': 'auto'}
{'C': 4.4588650039103985, 'degree': 2, 'gamma': 'scale'}
{'C': 1.9172075941288886, 'degree': 2, 'gamma': 'auto'}


### Random search  v2
z zawężonym przedziałem wartości C: [2, 4.5]  
oraz stopniem wielomianu z przedziału [2, 3, 4]

In [75]:
distributions = dict(C=uniform(loc=2, scale=2.5), gamma = ['scale', 'auto'], degree = [2, 3, 4]) 
clf = RandomizedSearchCV(svm_base, param_distributions = distributions, random_state=0, cv=3, n_iter = 20)
search = clf.fit(X1_train, y_train)

In [31]:
search_res2 = pd.DataFrame(search.cv_results_)

# Top 5 hiperparametrów najlepszych wyników
for i in range(5):
    print(search_res2.iloc[i, 7])

{'C': 3.372033759818312, 'degree': 3, 'gamma': 'scale'}
{'C': 3.5069084401791097, 'degree': 3, 'gamma': 'auto'}
{'C': 3.6147352826666403, 'degree': 2, 'gamma': 'auto'}
{'C': 4.229432501955199, 'degree': 2, 'gamma': 'scale'}
{'C': 2.9586037970644443, 'degree': 4, 'gamma': 'auto'}


### Porównanie błędu RMSE 3 najlepszych wyników oraz bazowego klasyfikatora

In [66]:
svm_top1 = SVC(C = 3.37, degree = 3, gamma = 'scale')
svm_top2 = SVC(C = 3.50, degree = 3, gamma = 'auto')
svm_top_search_1 = SVC(C = 2.74, degree = 2, gamma = 'auto')

In [67]:
svm_top_search_1.fit(X1_train, y_train)
y_hat = svm_top_search_1.predict(X1_test)
mse_top_search_1 = mean_squared_error(y_test, y_hat)
print("RMSE:", mean_squared_error(y_test, y_hat))

RMSE: 520770.364


In [61]:
svm_top1.fit(X1_train, y_train)
y_hat = svm_top1.predict(X1_test)
mse_top1 = mean_squared_error(y_test, y_hat)
print("RMSE:", mean_squared_error(y_test, y_hat))

RMSE: 94571.604


In [62]:
svm_top2.fit(X1_train, y_train)
y_hat = svm_top2.predict(X1_test)
mse_top2 = mean_squared_error(y_test, y_hat)
print("RMSE:", mean_squared_error(y_test, y_hat))

RMSE: 357699.868


### Podsumowanie

In [72]:
res_tab = pd.DataFrame()
res_tab["Classif"] = ['SVC top1', 'SVC top2','SVM top search1', 'SVC Base']
res_tab['RMSE'] = [mse_top1, mse_top2, mse_top_search_1, mse_base]
res_tab

,Classif,RMSE
0,SVC top1,94571.604
1,SVC top2,357699.868
2,SVM top search1,520770.364
3,SVC Base,242591.544


Najlepszy klasyfikator ma parametry: 
- `C`: 3.37
- `degree`: 3
- `gamma`: scale

Wartość błędu RMSE dla najlepszego klasyfikatora po podwójnym wyszukiwaniu metodą Randomized Search zmniejszyła się ponad 2.5 raza.
Z kolei dla kolejnych klasyfikatorów wartość błędu RMSE zwiększyła się w stosunku do bazowego klasyfikatora.


## Support Vector Machine - `Diabetes`

Poniżej wykorzystano ramkę danych `Diabetes` z pakietu Scikit Learn

In [76]:
from sklearn import datasets

In [77]:
print(datasets.load_diabetes().DESCR)

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, T-Cells (a type of white blood cells)
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, thyroid stimulating hormone
      - s5      ltg, lamotrigine
      - s6      glu, blood sugar level

Note: Each of these 10 feature va

In [78]:
y = datasets.load_diabetes().target
X = datasets.load_diabetes().data

In [79]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 45)

### Wyniki klasyfikacji dla bazowego klasyfikatora

In [86]:
svm_base3 = SVC()
svm_base3.fit(X_train, y_train)
y_hat = svm_base3.predict(X_test)
mse_diab_base = mean_squared_error(y_test, y_hat)
print('RMSE', mean_squared_error(y_test, y_hat))

RMSE 4679.036036036036


## Randomized Search
Skorzystamy z wcześniej wykorzystanego modelu `svm_base`

In [81]:
distributions = dict(C=uniform(loc=0, scale=10), gamma = ['scale', 'auto'], degree = [2, 3, 4,5,6])
clf = RandomizedSearchCV(svm_base, param_distributions = distributions, random_state=0, cv=3, n_iter = 20)
search = clf.fit(X_train, y_train)

In [82]:
search_res = pd.DataFrame(search.cv_results_)

# Top 5
for i in range(5):
    print(search_res.iloc[i, 7])

{'C': 5.4881350392732475, 'degree': 2, 'gamma': 'auto'}
{'C': 8.579456176227568, 'degree': 3, 'gamma': 'auto'}
{'C': 6.458941130666561, 'degree': 6, 'gamma': 'auto'}
{'C': 8.917730007820797, 'degree': 2, 'gamma': 'scale'}
{'C': 3.8344151882577773, 'degree': 2, 'gamma': 'auto'}


### Randomized Search v2
- C: 4-8
- degree: [2, 3, 4]
- gamma: ['auto', 'scale']

In [83]:
distributions = dict(C=uniform(loc=4, scale=4), gamma = ['scale', 'auto'], degree = [2, 3, 4]) 
clf = RandomizedSearchCV(svm_base, param_distributions = distributions, random_state=0, cv=3, n_iter = 30)
search = clf.fit(X_train, y_train)

In [84]:
search_res2 = pd.DataFrame(search.cv_results_)

# Top 5
for i in range(5):
    print(search_res.iloc[i, 7])

{'C': 5.4881350392732475, 'degree': 2, 'gamma': 'auto'}
{'C': 8.579456176227568, 'degree': 3, 'gamma': 'auto'}
{'C': 6.458941130666561, 'degree': 6, 'gamma': 'auto'}
{'C': 8.917730007820797, 'degree': 2, 'gamma': 'scale'}
{'C': 3.8344151882577773, 'degree': 2, 'gamma': 'auto'}


In [89]:
svm_top1 = SVC(C = 5.49, degree = 2, gamma = 'auto') 
svm_top2 = SVC(C = 8.57, degree = 3, gamma = 'auto')

In [90]:
svm_top1.fit(X_train, y_train)
y_hat = svm_top1.predict(X_test)
print('RMSE', mean_squared_error(y_test, y_hat))

RMSE 8984.162162162162


In [91]:
svm_top2.fit(X_train, y_train)
y_hat = svm_top2.predict(X_test)
print('RMSE', mean_squared_error(y_test, y_hat))

RMSE 8984.162162162162


Dla przeprowadzonego Randomized Search wyniki klasyfikacji nie poprawiły się.